In [4]:
import numpy as np
import pandas as pd
import os

df = pd.read_csv('MetObjects.csv')

In [6]:
df.describe()

,Object ID,Object Begin Date,Object End Date
count,448203.000000,4.482030e+05,4.482030e+05
mean,346272.873187,1.390817e+03,1.493446e+03
std,206914.987003,2.779816e+04,2.778712e+04
min,1.000000,-9.500000e+04,-3.500000e+04
25%,198808.500000,1.567000e+03,1.600000e+03
50%,349816.000000,1.800000e+03,1.850000e+03
75%,481764.500000,1.895000e+03,1.910000e+03
max,750830.000000,1.859186e+07,1.859186e+07


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 448203 entries, 0 to 448202
Data columns (total 43 columns):
Object Number              448203 non-null object
Is Highlight               448203 non-null bool
Is Public Domain           448203 non-null bool
Object ID                  448203 non-null int64
Department                 448203 non-null object
Object Name                445568 non-null object
Title                      416906 non-null object
Culture                    186518 non-null object
Period                     71882 non-null object
Dynasty                    23018 non-null object
Reign                      10817 non-null object
Portfolio                  20370 non-null object
Artist Role                259909 non-null object
Artist Prefix              88928 non-null object
Artist Display Name        261111 non-null object
Artist Display Bio         224064 non-null object
Artist Suffix              10212 non-null object
Artist Alpha Sort          261088 non-null object


In [8]:
df['Object Name'].value_counts()

Print                                              88972
Photograph                                         28434
Drawing                                            24966
Book                                               13412
Fragment                                            9405
Piece                                               8634
Negative                                            6258
Painting                                            5887
Baseball card, print                                5208
Bowl                                                3535
Figure                                              3089
Baseball card                                       3046
Polaroid                                            2706
Vase                                                2669
Dress                                               2468
Baseball card, photograph                           2141
Books Prints Ornament & Architecture                2127
Ensemble                       

In [9]:
public_domain = df.loc[df['Is Public Domain'] == True]
objects = public_domain['Object Name'].value_counts()
objects

Print                                                  26618
Drawing                                                14165
Fragment                                                8050
Photograph                                              7682
Piece                                                   6425
Painting                                                3537
Bowl                                                    2770
Figure                                                  2244
Vase                                                    2103
Books Prints Ornament & Architecture                    2049
Dish                                                    1647
Plate                                                   1593
Panel                                                   1323
Bottle                                                  1303
Medal                                                   1278
Ornament                                                1093
Jar                     